In [1]:
# Install stuff
!pip install transformers==4.33.0 accelerate optimum scipy auto-gptq -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 25.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 32.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import dependencies
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

In [3]:
# Specify model name 
model_name = "TheBloke/Falcon-180B-Chat-GPTQ"

In [4]:
# Load quantized model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    revision="main", cache_dir='workspace/models/')

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
# Make it better 
model.to_bettertransformer()

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


FalconForCausalLM(
  (transformer): FalconModel(
    (word_embeddings): Embedding(65024, 14848)
    (h): ModuleList(
      (0-79): 80 x FalconDecoderLayer(
        (self_attention): FalconAttentionLayerBetterTransformer(
          (maybe_rotary): FalconRotaryEmbedding()
          (query_key_value): QuantLinear()
          (dense): QuantLinear()
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): FalconMLP(
          (act): GELU(approximate='none')
          (dense_4h_to_h): QuantLinear()
          (dense_h_to_4h): QuantLinear()
        )
        (ln_attn): LayerNorm((14848,), eps=1e-05, elementwise_affine=True)
        (ln_mlp): LayerNorm((14848,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_f): LayerNorm((14848,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=14848, out_features=65024, bias=False)
)

In [6]:
# Load up tokenizer 
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [7]:
# Setup a prompt 
prompt = """Explain in a short paragraph quantum field theory to a high-school student."""

# Pass the prompt to the tokenizer
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
# Setup the text streamer 
streamer = TextStreamer(tokenizer, skip_prompt=True)

In [11]:
del inputs['token_type_ids']

In [8]:
# Actually run the thing
output = model.generate(**inputs,
                        streamer=streamer,
                        use_cache=True, 
                        max_new_tokens=60)

ValueError: The following `model_kwargs` are not used by the model: ['token_type_ids'] (note: typos in the generate arguments will also show up in this list)